In [2]:
import wrds
file_key = open("../../keys/wrds_username.txt","r")
USERNAME = file_key.read()
file_key.close()

db = wrds.Connection(wrds_username=USERNAME)
#db.create_pgpass_file()

Loading library list...
Done


In [3]:
import pandas as pd
import numpy as np
from datetime import date

# Fama Bliss

In [4]:
db.describe_table(library='crsp_m_treasuries', table='tfz_mth_fb')

SQL_QUERY = f'select * from crsp_m_treasuries.tfz_mth_fb'
fb_raw = db.raw_sql(SQL_QUERY)
fb = fb_raw.rename(columns={'mcaldt':'date','tmnomprc':'price','tmytm':'yld'})
fb = fb.pivot_table(values='price',index='date',columns='kytreasnox')
fb.rename(columns={2000047:1, 2000048:2, 2000049:3, 2000050:4, 2000051:5},inplace=True)
fb.columns.name = 'maturity'
fb

Approximately 4350 rows in crsp_m_treasuries.tfz_mth_fb.


maturity,1,2,3,4,5
date,,,,,
1952-06-30,98.177763,96.253133,94.246713,92.634134,90.418295
1952-07-31,98.094756,96.031453,93.931226,92.308865,89.851576
1952-08-29,98.057808,95.974988,93.781939,92.145776,89.571687
1952-09-30,98.031188,95.909789,93.755108,92.096104,89.167641
1952-10-31,98.110769,95.974490,93.758273,91.929019,89.724841
...,...,...,...,...,...
2024-07-31,95.442882,91.930210,88.676493,85.502712,82.412153
2024-08-30,95.761807,92.544654,89.401025,86.226586,83.196425
2024-09-30,96.148197,93.062254,89.977548,86.813966,83.795135


# Save to Excel

In [5]:
DATE = fb.index[-1]
outfile = f'../data/famabliss_strips_{DATE}.xlsx'
with pd.ExcelWriter(outfile) as writer:
    fb.to_excel(writer, sheet_name= 'prices', index=True)

***